In [1]:
import numpy as np
import pandas as pd
import pickle
import random
import joypy

import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from cfg_global import project_folder_path
os.chdir(current_wd)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import seaborn as sns

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'

In [8]:
dataset_folder = project_folder_path + 'results\\synthetic\\dataset\\'
results_folder_zeros = project_folder_path + 'results\\synthetic\\results\\fixed_params_sttc_vs_zeros\\'
results_folder = project_folder_path + 'results\\synthetic\\results\\fixed_params\\'
fig_folder = project_folder_path + 'results\\synthetic\\fig_drafts_paper\\'

In [5]:
save_fig = False

### Load data

In [ ]:
with open(dataset_folder + '1_spike_trains_3params_var_len600sec_100000.pkl','rb') as f:
    data = pickle.load(f)

spike_trains = data['spike_trains']
alphas = data['alphas']
fr_values = data['fr_values']
taus_ms = data['tau_ms']
duration_ms = data['duration_ms']

print(f'n spike trains {len(spike_trains)}, len {spike_trains[0][-1]/1000}, duration_ms {duration_ms}')

fs = 1000

In [26]:
with open(results_folder + 'sttc_trial_avg_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_avg_dict_ = pickle.load(f)
    sttc_trial_avg_dict = dict(list(sttc_trial_avg_dict_.items())[:100])

with open(results_folder + 'sttc_trial_concat_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_concat_dict_ = pickle.load(f)
    sttc_trial_concat_dict = dict(list(sttc_trial_concat_dict_.items())[:100])

with open(results_folder_zeros + 'sttc_trial_avg_50ms_dict.pkl', "rb") as f:
    sttc_trial_avg_zeros_dict = pickle.load(f)

with open(results_folder_zeros + 'sttc_trial_concat_50ms_dict.pkl', "rb") as f:
    sttc_trial_concat_zeros_dict = pickle.load(f)

print(f'len sttc_trial_avg_dict {len(sttc_trial_avg_dict)}')
print(f'len sttc_trial_concat_dict {len(sttc_trial_concat_dict)}')
print(f'len sttc_trial_avg_zeros_dict {len(sttc_trial_avg_zeros_dict)}')
print(f'len sttc_trial_concat_zeros_dict {len(sttc_trial_concat_zeros_dict)}')

len sttc_trial_avg_dict 100
len sttc_trial_concat_dict 100
len sttc_trial_avg_zeros_dict 100
len sttc_trial_concat_zeros_dict 100


### Make summary dfs

In [13]:
def get_trials_plot_df(trial_dict_, method_, n_iteration_=None):
    records = []
    for unit_id, data in trial_dict_.items():
        taus = data['taus']
        acfs = data['acf']

        if n_iteration_ is not None:
            # only one trial per unit
            idx = n_iteration_[unit_id]
            taus_to_iter = [(taus[idx], acfs[idx])]
        else:
            # all trials for this unit
            taus_to_iter = zip(taus, acfs)

        for tau_dict, acf_array in taus_to_iter:
            records.append({
                'unit_id': unit_id,
                'tau': tau_dict['tau'],
                'fit_r_squared': tau_dict['fit_r_squared'],
                'method': method_,
            })

    df = pd.DataFrame.from_records(records)
    df['tau_ms'] = df['tau'] * 50

    nan_count = df.isna().any(axis=1).sum()
    if nan_count > 0:
        print(f'N rows with NaNs {nan_count}')

    return df

In [27]:
# for trial based measures one realization of trials is taken
random_trials_impl = np.zeros(len(sttc_trial_concat_dict)).astype(int)

sttc_trial_concat_plot_df = get_trials_plot_df(sttc_trial_concat_dict, 'sttc_trial_concat', random_trials_impl)
sttc_trial_avg_plot_df = get_trials_plot_df(sttc_trial_avg_dict, 'sttc_trial_avg', random_trials_impl)

sttc_trial_concat_plot_0_df = get_trials_plot_df(sttc_trial_concat_zeros_dict, 'sttc_trial_concat_0', random_trials_impl)
sttc_trial_avg_plot_0_df = get_trials_plot_df(sttc_trial_avg_zeros_dict, 'sttc_trial_avg_0', random_trials_impl)

In [30]:
sttc_trial_concat_plot_0_df

,unit_id,tau,fit_r_squared,method,tau_ms
0,0,4.437984,0.381404,sttc_trial_concat_0,2.218992e+02
1,1,25311.781035,0.409512,sttc_trial_concat_0,1.265589e+06
2,2,73195.222647,0.153249,sttc_trial_concat_0,3.659761e+06
3,3,0.351694,0.266793,sttc_trial_concat_0,1.758471e+01
4,4,1.554160,0.267689,sttc_trial_concat_0,7.770799e+01
...,...,...,...,...,...
95,95,1.620555,0.683022,sttc_trial_concat_0,8.102773e+01
96,96,1.984685,0.029179,sttc_trial_concat_0,9.923425e+01
97,97,0.102099,0.009460,sttc_trial_concat_0,5.104956e+00
98,98,3.560763,0.657158,sttc_trial_concat_0,1.780382e+02


In [31]:
sttc_trial_concat_plot_df

,unit_id,tau,fit_r_squared,method,tau_ms
0,0,4.437984,0.381404,sttc_trial_concat,2.218992e+02
1,1,25311.781035,0.409512,sttc_trial_concat,1.265589e+06
2,2,73195.222647,0.153249,sttc_trial_concat,3.659761e+06
3,3,0.351694,0.266793,sttc_trial_concat,1.758471e+01
4,4,1.554160,0.267689,sttc_trial_concat,7.770799e+01
...,...,...,...,...,...
95,95,1.620555,0.683022,sttc_trial_concat,8.102773e+01
96,96,1.984685,0.029179,sttc_trial_concat,9.923425e+01
97,97,0.102099,0.009460,sttc_trial_concat,5.104956e+00
98,98,3.560763,0.657158,sttc_trial_concat,1.780382e+02


In [ ]:
tau_plot_long_df = pd.concat([pearsonr_sans_negr_plot_df, 
                              sttc_concat_sans_negr_plot_df, 
                              sttc_avg_sans_negr_plot_df])
tau_plot_long_df['log_tau_ms'] = np.log10(tau_plot_long_df['tau_ms'])
tau_plot_long_df.reset_index(inplace=True, drop=True)

print(f'len tau_plot_long_df {len(tau_plot_long_df)}')

tau_plot_long_df.head(3)

In [ ]:
tau_plot_wide_pearsonr_concat_df = pearsonr_sans_negr_plot_df.merge(sttc_concat_sans_negr_plot_df, 
                                                                    suffixes=('_pearsonr', '_sttc_concat'), 
                                                                    on=['unit_id','fr', 'alpha', 'tau_ms_true'], how='left')
tau_plot_wide_pearsonr_concat_df.reset_index(inplace=True, drop=True)

print(f'len tau_plot_wide_pearsonr_concat_df {len(tau_plot_wide_pearsonr_concat_df)}')

tau_plot_wide_pearsonr_concat_df.columns

In [ ]:
tau_plot_wide_pearsonr_avg_df = pearsonr_sans_negr_plot_df.merge(sttc_avg_sans_negr_plot_df, 
                                                                    suffixes=('_pearsonr', '_sttc_avg'), 
                                                                    on=['unit_id','fr', 'alpha', 'tau_ms_true'], how='left')
tau_plot_wide_pearsonr_avg_df.reset_index(inplace=True, drop=True)

print(f'len tau_plot_wide_pearsonr_avg_df {len(tau_plot_wide_pearsonr_avg_df)}')

### Plots

In [ ]:
color_pearson_trail_avg = '#f4a91c' 
color_sttc_trail_avg =  '#a49fce' 
color_sttc_trail_concat = '#955da2' 

colors = [color_pearson_trail_avg, color_sttc_trail_avg, color_sttc_trail_concat]

In [ ]:
#tau_plot_long_df = pd.read_pickle()

In [ ]:
tau_plot_long_df.groupby(by='method', as_index=False)['tau_diff_rel'].median()

In [ ]:
tau_plot_long_df['tau_diff_rel_log10'] = np.log10(tau_plot_long_df['tau_diff_rel'])

In [ ]:
sns.kdeplot(
    data=tau_plot_long_df, 
    x="tau_diff_rel_log10", 
    hue="method",
    fill=True, 
    alpha=0.3,
    palette=colors
)